# 3 Tensor

In [32]:
import torch
import numpy as np 

# 张量初始化的方法
# 直接生成张量
data = [[1,2],[3,4]]
x_data = torch.tensor(data)

# numpy数组转化
np_array = np.array(data)
x_np=torch.from_numpy(np_array)

# 通过已有的张量生成新的张量
x_ones = torch.ones_like(x_data)
x_rand = torch.rand_like(x_data,dtype = torch.float)

# 通过制定数组的维度生成张量
shape = (2,3)
rand_tensor = torch.rand(size=shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

# 张量的属性
tensor = torch.rand(3,4)
print(tensor.shape,tensor.dtype,tensor.device)

# 张量运算
# 将tensor导入gpu内运行
if torch.cuda.is_available():
    tensor = tensor.to('cuda')
# 张量的索引和切片-与numpy数组完全一致
tensor = torch.ones(4,4)
tensor[:,1]=0
print(tensor)

# 张量的拼接_在某个内层已有的维度上延伸
t1 = torch.cat([tensor,tensor,tensor],dim=1)
print(t1)

# 张量的拼接_创建一个新的维度，组装原来的tensor。在某个维度上组装原来的tensor。原来的tensor不会被破坏
t2 = torch.stack((tensor,),dim=1)
print(t2)

# 张量的乘积和矩阵乘法
# 逐个元素相乘
tensor.mul(tensor)
tensor*tensor
# 矩阵乘法
tensor.matmul(tensor.T)
tensor @ tensor.T

# 自赋值运算 _自赋值。否则返回运算后的值。
tensor.add_(5)
print(tensor)

# tensor与numpy的相互转换。而且numpy与tensor公用同样的地址的值
np_array = tensor.numpy()
print(np_array)
tensor.add_(3)
print(np_array)

torch.Size([3, 4]) torch.float32 cpu
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])
tensor([[[1., 0., 1., 1.]],

        [[1., 0., 1., 1.]],

        [[1., 0., 1., 1.]],

        [[1., 0., 1., 1.]]])
tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])
[[6. 5. 6. 6.]
 [6. 5. 6. 6.]
 [6. 5. 6. 6.]
 [6. 5. 6. 6.]]
[[9. 8. 9. 9.]
 [9. 8. 9. 9.]
 [9. 8. 9. 9.]
 [9. 8. 9. 9.]]


## 4 torch.autograd 

In [40]:
# 正向传播和方向传播的用法
import torch,torchvision
model = torchvision.models.resnet18(pretrained=True)

data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)
# print(data)
prediction = model(data)
# print(prediction.type)
loss = (prediction-labels).sum()
loss.backward()
optim = torch.optim.SGD(model.parameters(),lr=1e-2,momentum=0.9)
optim.step()

In [56]:
# autograd实现微分
# tensor 能够通过autograd自动梯度。进行梯度下降分析。即记住运算过程。
# 每个张量都有自己的grad属性。用来存储本张量的梯度下降值。
import torch

a = torch.tensor([2.,3.],requires_grad=True)
b = torch.tensor([6.,4.],requires_grad=True)


Q = 3*a**3 -b**2

# 构建一个梯度运算的图（树）。
print(a.grad)
print(b.grad)

# Q.sum().backward()
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad)

print(a.grad)
print(b.grad)

print(9*a**2 == a.grad)
print(-2*b == b.grad)

None
None
tensor([36., 81.])
tensor([-12.,  -8.])
tensor([True, True])
tensor([True, True])


> 计算图.自动维护一个计算图。输入节点为叶节点。输出节点为根节点。的树。

## 5 神经网络

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f

class Net(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(16*6*6,120)
        self